# Инкапсуляция 

Под инкапсуляцией чаще всего понимают связывание данных и методов, работающих с этими данными, в единую сущность. В рамках этого раздела будет раскрыт механизм создания атрибутов и методов класса. 

## Атрибуты 

### Атрибуты объекта `vs` атрибуты класса

Напомним, что класс является типом для своих экземпляром, но в рамках `python` и сам является объектом. Это приводит к тому, что в `python` различают два типа атрибутов: атрибуты класса и атрибуты объектов. 
- **атрибут объекта** --- атрибут (или поле) в классическом понимании, может принимать разные значения для разных экземпляров; 
- **атрибут класса** --- "статический" атрибут, который принадлежит объекту объявления класса, который однако доступен и каждому экземпляру этого класса; 

Для иллюстрации создадим пустой класс и два его экземпляра.

In [41]:
class A:
    pass

a1, a2 = A(), A()

В большинстве случаев `python` позволяет цеплять атрибуты к объектам налету, т.е. создавать новые атрибуты у уже существующих объектов. Для этого используется синтаксис **`object.attribute = value`**.

Создадим с помощью такого синтаксиса атрибут `static_attribute` у класса `A`.

In [42]:
A.static_attribute = "shared"
print(A.static_attribute)

shared


Видим, что у класса `A` теперь появился атрибут `static_attribute` со значением `"shared"`. 

Посмотрим, что получится, если попробовать спросить атрибут `static_attribute` у экземпляров `a1` и `a2` класса `A`.

In [39]:
print(a1.static_attribute, a2.static_attribute)

shared shared


Видим, что вместо ошибки мы получили тоже самое значение `"shared"`. Дело в том, что запись **`object.attribute`** вынуждает поиск атрибута **`attribute`** у объекта **`object`** в `runtime`, при этом, если у объекта **`object`** такой атрибут отсутствует, то на этом поиск не прекращается, а сначала поиск этого же самого атрибута делегируется объекту **`type(object)`**, т.е. соответствующему объекту объявления класса. Только если и объект объявления класса ответит, что такого атрибута нет, то возбуждается исключение [AttributeError](https://docs.python.org/3/library/exceptions.html#AttributeError).

```{tip}
Описанное выше поведение справедливо для большинства объектов. Возможно, однако, повлиять на механизм поиска атрибутов, перегрузив специальные методы. В современном `python` редко приходится прибегать к таким трюкам, поэтому они здесь не приводятся. 
```

Для контраста создадим атрибут `x` у одного из экземпляров класса `A`. 

In [43]:
a1.x = 3.14
print(a1.x)             # success:  3.14
print(a2.x)             # error:    AttributeError   

3.14


AttributeError: 'A' object has no attribute 'x'

Видим, что запрос атрибута `x` у объекта `a1` возвращает новое значение `3.14`, а в случае с `a2` возбуждается ошибка `AttributeError`. Т.е. мы получили другой тип атрибута, значение которого не разделяется между классом и всеми его экземплярами. Если создать атрибут `x` у экземпляра `a2`, то значения атрибута `x` не претерпит никаких изменений.  

In [44]:
a2.x = 2.71
print(a1.x, a2.x)

3.14 2.71


### Создание атрибутов 

Выше оба типа атрибутов создавались "на лету", т.е. сначала создавались объекты, а потом у уже существующих объектов создавались атрибуты. При этом тип атрибута определялся тем, у какого объекта этот атрибут был создан. Такой подход не является общепринятым, так как он полон недостатков. 

Во-первых, код создания атрибутов объекта находится за пределами класса этих объектов, а значит класс перестаёт контролировать поведение своих экземпляров: чтобы понять, как обращаться с объектами этого класса не достаточно посмотреть код внутри этого класса, необходимо также проследить весь цикл жизни объекта в сопровождающем коде. 

Во-вторых, создавая атрибуты "на лету", легко допустить ошибку и получить объекты одного класса, имеющие при этом разный набор атрибутов, что может привести к многим часам отладки в будущем. В примере выше в определенный момент времени существовали два экземпляра одного и того класса, у одного из которых был атрибут `x`, а другого нет. Такая ситуация противоречит принципам ООП. 

Для создания атрибутов обоих типов в `python` предусмотренны общепринятые места: 
- атрибуты класса создаются в теле класса;
- атрибуты экземпляров создаются либо на этапе создания экземпляра (специальный метод `__new__`), либо на этапе его инициализации (специальный метод `__init__`). 

Ниже приводится объявление класса `A` с статическим атрибутом `static_attribute` со значением `shared`, экземпляры которого имеют атрибут `x`, значение которого уникально для каждого экземпляра. К последнему вернемся позже, т.к. нам пока не хватает знаний о методах класса. 

In [2]:
class A:
    static_attribute = "shared"
    
    def __init__(self, x):
        self.x = x


## Методы 

Обычные методы в `python` объявляются внутри объявления класса в виде обычных функций с небольшой поправкой: в списке параметров в начало всегда добавляется дополнительный параметр `self`, который ссылается на объект, вызвавший этот метод. Этот параметр играет такую же роль, что и ключевое слово `this` в языке `C++`, но в `python` требуется в явном виде указывать этот параметр в списке параметров. 


В ячейке ниже определяется класс `Conscious` с единственным методом `id`, который возвращает адрес памяти объекта, который его вызывал. Далее создаётся два экземпляра этого класса и от них вызывается этот метод `id`.

In [6]:
class Conscious:
    def id(self):
        return id(self)

c1, c2 = Conscious(), Conscious()
print(c1.id(), c2.id())

2330800289824 2330800279120


На самом деле вызовы `c1.id()` и `c2.id()` являются сокращенной формой записи для выражений ниже.

In [7]:
print(Conscious.id(c1), Conscious.id(c2))

2330800289824 2330800279120


Если объект **`object`** является экземпляром класса **`SomeClass`**, а метод **`method`** определен внутри этого класса, то запись **`object.method(...)`** разворачивается в виде **`SomeClass.method(object, ...)`**, т.е. методы действительно вызываются в той форме, в какой они объявлены: при вызове метода через объект `python` автоматически находит нужный метод в классе и подставляет вызвавший объект в качестве первого параметра. 

```{tip}
При следовании принципам ООП нормальный метода класса так или иначе должен взаимодействовать с одним или более экземпляром этого класса. Иначе справедлив вопрос, а должен ли этот метод принадлежать классу, если он никак не взаимодействует с его экземплярами? В подавляющем большинстве случаев ответ *нет*, и этот метод целесообразно вынести за пределы класса в виде обычной функции.

Тем не менее иногда такую функцию все же хочется "упаковать" внутрь класса, например, чтобы подчеркнуть семантическую близость этой функции к этому классу. Такие функции называют **статическими методами** и для их создания в `python` предусмотрен специальный декоратор [staticmethod](https://docs.python.org/3/library/functions.html#staticmethod), который "выключает" подстановку ссылки экземпляр в список параметров.
```

Важно понимать, что `python` подставляет экземпляр именно первым параметром. 

```{tip}
На самом деле `python` не навязывает использовать в качестве имени первого параметра именно `self`, но в сообществе программистов сложилась традиция называть его именно так, и я вам рекомендую от этой традиции без особых на то причин не отказываться. 
```

## Создание объектов

### `__new__` **vs** `__init__`

Пусть интерпретатор встречает выражение **`x = SomeClass(...)`**, где **`SomeClass`** --- некоторый тип. Тогда запускается механизм создание экземпляра класса **`SomeClass`**, который всегда состоит из двух этапов.

1. Сначала вызывается метод **`SomeClass.__new__(...)`**, который отвечает за **создание** объекта с нуля. 
2. Затем вызывается метод **`__init__`** у созданного объекта. Этот метод отвечает за **инициализацию** объекта: чаще всего это создание атрибутов. Пусть, **`t`** --- созданный на предыдущем шаге объект, тогда вызывается метод **`t.__init__(...)`**, что эквивалентно вызову **`SomeClass.__init__(t, ...)`**. 

```{tip}
Вы могли заметить, что методы **`__new__`** и **`__init__`** начинаются и заканчиваются с двух символов нижнего подчеркивания **'_'**. В `python` все методы с именами в таком стиле (**`__method_name__`**) считаются специальными. Они всегда играют какую-то специальную  роль (например, создание и инициализация), и постоянно вызываются самим интерпретатором **`python`** под капотом в служебных целях. Перегрузив эти методы, можно встроить объекты пользовательских типов в экосистему `python`. Подробнее о специальных методах позже (см. [special_methods]()).
```

Если выполнение обоих этих методов завершается безошибочно, то созданный и инициализированный объект возвращается вызывающему коду (**`x = t`** в нашем случае). 

Перегрузка метода **`__new__`** --- материал для продвинутых программистов, требующий глубоких знаний о том, как `python` работает изнутри. К счастью, в подавляющем большинстве ситуаций можно обойтись перегрузкой метода **`__init__`**. Поэтому, далее речь пойдет только об этом методе. 


### Инициализация объекта

Итак, если вы не перегрузили метод **`__new__`**, то создаётся объект без атрибутов (за исключением служебных). Далее вызывается метод **`__init__`**, основная цель которого --- наделить этот объект атрибутами с правильными значениями. 

В качестве примера рассмотрим класс `A`, объекты которого должны обладать атрибутом `x`. 

In [4]:
class A:
    def __init__(self, x):
        self.x = x

a1 = A(3.14)
a2 = A(2.71)
print(a1.x, a2.x)

3.14 2.71


Видим, что все работает, как задумывалось. Изучим метод `__init__` чуть внимательнее. 

1. Метод `__init__` принимает два параметра: `self` и `x`.
2. Параметр `self` --- ссылка на экземпляр, от которого этот метод был вызван.
3. Параметр `x` используется для того, чтобы передавать необходимое значение атрибута при создании экземпляра. 
4. `self.x = x` --- единственная инструкция в теле метода, которая создаёт атрибут `self.x` и связывает его со значением, переданным в конструктор.

Заметьте, что как и в прежних примерах, атрибуты создаются у уже существующего объекта. Но большая разница заключается в том, что при создании объекта вызов метода `__init__` гарантирован, а значит можно обеспечить необходимый набор атрибутов у каждого объекта.


## Пример: вектор трехмерного пространства евклидова пространства

Для закрепления материала реализуем класс `Vector2D`, хранящий в себе координаты в некотором ортонормированном базисе, и предоставляющий метод `norm`, вычисляющий длину этого вектора.

In [2]:
from math import hypot

class Vector2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def norm(self):
        return hypot(self.x, self.y)


```{admonition} Упражнение 1
Убедитесь, что вам понятен код этого примера. Создайте пару экземпляров класса `Vector3D`. Протестируйте их поведение.
```

## Альтернативные конструкторы. `Classmethod`

В `python` нельзя перегружать функции, а т.к. методы класса --- это обычные функции, объявленные внутри класса, то и их перегружать тоже нельзя. Из-за этого у класса может быть всего один метод `__init__`. 

Нередко такую проблему решают с помощью декоратора [classmethod](https://docs.python.org/3/library/functions.html#classmethod). Методы, помеченные этим декоратором, работают иначе: им не передаётся первым параметром ссылка на экземпляр, а передаётся ссылка на класс, которой принято давать имя `cls`. 

Код в ячейке ниже иллюстрирует разницу между обычным методом и `classmethod`.

In [6]:
class A:
    def common_method(self):
        print(self)

    @classmethod
    def class_method(cls):
        print(cls)



a = A()
a.class_method()
A.class_method()
a.common_method()
A.common_method()

<class '__main__.A'>
<class '__main__.A'>


TypeError: A.common_method() missing 1 required positional argument: 'self'

Обратите внимание, что метод `class_method` можно вызывать и от экземпляра `a`, и от самого класса `A`: эффект остаётся одинаковым. Про обычный метод `common_method` такого не скажешь, т.к. при вызове через экземпляр в список аргументов автоматические вставляется ссылка на вызвавший экземпляр, а при вызове через объект класса такого не происходит. Однако обычно классовые методы вызывают через объект класса, т.к. они не получают ссылки на экземпляр.

В качестве примера модифицируем предыдущий пример с вектором `Vector2D`. Предположим, мы хотим иметь возможность создавать экземпляры этого класса, используя вместо декартовых координат полярные. Для этого можно добавить метод `from_polar` и пометить его декоратором `classmethod`.

In [8]:
from math import hypot, cos, sin

class Vector2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def norm(self):
        return hypot(self.x, self.y)

    @classmethod
    def from_polar(cls, rho, phi):
        return cls(rho*cos(phi), rho*sin(phi))

```{note}
Внутри метода `from_polar` имя `cls` ссылается на класс `Vector2D`. Можно было бы возвращать из этого метода выражение `Vector2D(rho*cos(phi), rho*sin(phi))`, но такой подход менее гибкий, если появляется иерархия наследования: `cls(rho*cos(phi), rho*sin(phi))` корректно отработает и для производных классов, а `Vector2D(rho*cos(phi), rho*sin(phi))` будет возвращать экземпляр класса `Vector2D`, даже если метод был вызван от производного класса. 
```

Теперь можно создать вектор на основе полярных координат этим методом.

In [10]:
from math import pi 

v = Vector2D.from_polar(1, pi/6)
print(v.x, v.y)

0.8660254037844387 0.49999999999999994


## Статические методы. `staticmethod`

В `python` существует ещё один декоратор [static](https://docs.python.org/3/library/functions.html#staticmethod), который отменяет подстановку ссылки на экземпляр, от которого был вызван метод, в список аргументов. 

Ниже иллюстрируется поведение статических методов. Обратите внимание, что они работают одинаково, если их вызвать через экземпляр и через объект класса.

In [11]:
class A:
    @staticmethod
    def static_method():
        print("static")

a = A()
a.static_method()
A.static_method()

static
static


Область применения таких методов --- спорная тема. Статические методы ведут себя как обычные функции, но зачем-то помещенные внутрь класса. Многие настаивают, что такие методы и должны реализовываться в виде обычных функций снаружи класса. Единственный довод в пользу статических методов, с которым соглашается автор курса, заключается в том, что помещение статического метода внутрь класса подчеркивает семантическую близость этой функции к этому классу. 